In [9]:
# This code will read the tokens from tokenized_complete.txt, perform sentiment analysis on each token, and write the results to sentiment_results.txt. Each line in sentiment_results.txt will be a dictionary representing the sentiment scores of a token.Please note that this is a very basic example. The SentimentIntensityAnalyzer is more effective when used on larger pieces of text (like sentences or paragraphs) rather than individual tokens. Also, this code assumes that your tokens are separated by spaces. If your tokens are separated by something else, you’ll need to modify the split() function accordingly. 

import nltk, csv
import re
from nltk.sentiment import SentimentIntensityAnalyzer

# Initialize the SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

# Initialize an empty dictionary to store results
results = {}

# Initialize novel_name to None
novel_name = None

# Define a regex pattern for the novel titles
pattern = re.compile(r'\[(\w+)\]')

# Open the input file
with open('../output/merged_titles.txt', 'r') as f:
    # Iterate over each line in the file
    for line in f:
        # Remove leading/trailing whitespace
        line = line.strip()
        # If the line is a novel title (matches the regex pattern)
        match = pattern.match(line)
        if match:
            # If a novel has been processed, perform sentiment analysis on its entire text
            if novel_name:
                sentiment = sia.polarity_scores(' '.join(results[novel_name]))
                results[novel_name] = sentiment
            # Extract the new novel name and initialize its results list
            novel_name = match.group(1)
            results[novel_name] = []
        # If the line is part of a novel
        elif novel_name:
            # Add the line to the novel's results list
            results[novel_name].append(line)

# If a novel has been processed, perform sentiment analysis on its entire text
if novel_name:
    sentiment = sia.polarity_scores(' '.join(results[novel_name]))
    results[novel_name] = sentiment

# Open the output file
with open('../output/merged_sentiment_results.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    # Write the headers to the file
    writer.writerow(['novel_name', 'neg', 'neu', 'pos', 'compound'])
    # Iterate over each novel and its results
    for novel_name, sentiment in results.items():
        # Write the novel name and sentiment result to the file
        writer.writerow([novel_name] + list(sentiment.values()))


In [8]:
# Visualise the results: Import necessary libraries (pandas, matplotlib); Load the `merged_sentiment_results.txt` file into a pandas DataFrame; Extract the novel titles using the provided regex and add them as a new column in the DataFrame; Calculate the average sentiment for each novel; Plot the average sentiment for each novel using a bar chart or line graph.

import pandas as pd
import matplotlib.pyplot as plt
import re

# Load the data
df = pd.read_csv('../output/merged_sentiment_results.csv', sep="\t", header=None)

# Extract novel titles
df['title'] = df[0].apply(lambda x: re.search(r'\[(\w+)\]', x).group(1) if re.search(r'\[(\w+)\]', x) else '')

# Calculate average sentiment
average_sentiment = df.groupby('title')[1].mean()

# Plot
plt.figure(figsize=(10,5))
average_sentiment.plot(kind='bar')
plt.ylabel('Average Sentiment')
plt.show()


KeyError: 'Column not found: 1'